In [ ]:
# !pip install numpy==1.26.4
# !pip install pandas==2.2.2
# !pip install scikit-learn==1.5.1
# !pip install scipy==1.14.1
# !pip install statsmodels==0.14.2
# !pip install joblib==1.4.2
# !pip install threadpoolctl==3.5.0
# !pip install lightgbm==4.6.0
# !pip install catboost==1.2.3

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pytorch_tabular import TabularModel
from pytorch_tabular.models import (
CategoryEmbeddingModelConfig,
FTTransformerConfig,
TabNetModelConfig,
GANDALFConfig,
)
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.stacking import StackingModelConfig
# from pytorch_tabular.utils import make_mixed_dataset

from sklearn.preprocessing import LabelEncoder, FunctionTransformer, QuantileTransformer, MultiLabelBinarizer

from sklearn.impute import SimpleImputer

from sklearn.model_selection import KFold

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

import random

import preprocessing

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.utils.class_weight import compute_class_weight

from embedding import TabularPipeline
from eval_metric import competition_metric, f1_score, weighted_brier_score

import re

## CategoryEmbedding Model

In [2]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.categorical_encoders import CategoricalEmbeddingTransformer
from pytorch_tabular.models.common.heads import LinearHeadConfig

In [3]:
import sys
sys.path.append("../../")
from new_lgbm_process import all_process

seed = 333

train_path = "../../data/train.csv"
test_path = "../../data/test.csv"

train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])
train, test = all_process(train, test)
print(train.shape, test.shape)

(126244, 33) (54412, 32)


In [4]:
cat_cols = [col for col in train.columns if pd.api.types.is_object_dtype(train[col])]
numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 확률']

print(f'수치형 변수: {len(numeric_cols)}개 \n{numeric_cols}')
print(f'범주형 변수: {len(cat_cols)}개 \n{cat_cols}')
print(train.shape, test.shape)

수치형 변수: 21개 
['배란 자극 시술 여부', '단일 배아 이식 여부', '불임 원인 - 난관 질환', '불임 원인 - 배란 장애', '불임 원인 - 남성 요인', '불임 원인 - 자궁내막증', '불임 원인 - 불명확', '이전 IVF 시술 횟수', '이전 DI 시술 횟수', '이전 총 임신 횟수', '이전 총 임신 성공 횟수', '이식된 배아 수', '미세주입(ICSI) 배아 이식 수', '해동 난자 사용 여부', '신선 난자 사용 여부', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '착상 전 PGD 시행 여부', '착상 전 PGS 시행 여부', '배아 이식 후 경과일']
범주형 변수: 11개 
['환자 시술 당시 나이', '총 생성 배아 수', '저장된 배아 수', '해동된 배아 수', '채취된 신선 난자 수', '수정 시도된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '시술유형_통합']
(126244, 33) (54412, 32)


## Categorical Embedding Transformer
- embedding_dim = min(50, (num_categories + 1) // 2)
    - 범주가 4개 → 임베딩 dim = 2
    - 범주가 10개 → 임베딩 dim = 5
    - 범주가 200개 → 임베딩 dim = 50 (최대값 제한)
    
## Embedding+LGBM
- nsplit=5 : 0.739812
- nsplit=10 : 

In [5]:
seed = 333
kf = KFold(n_splits=5, shuffle=True, random_state=seed)

# 학습/평가 데이터 로드 (ID 칼럼 제거)
train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

# 평가 지표들을 저장할 리스트 초기화
f1_scores = []       # Custom F1 from eval_metric
wbs_scores = []      # Weighted Brier Score
comp_scores = []     # Combined metric (competition_metric)

# 최종 예측값 저장용
test_preds_lgbm = []

# StratifiedKFold 대신 일반 KFold 사용 (타겟이 연속값인 경우 사용 가능)
for fold, (train_idx, val_idx) in enumerate(kf.split(train, train['임신 성공 확률'])):
    
    # 현재 fold의 train/validation 데이터 분할
    train_fold = train.iloc[train_idx].copy().reset_index(drop=True)
    val_fold = train.iloc[val_idx].copy().reset_index(drop=True)    
    
    train2_fold = train_fold.copy()
    test_fold = test.copy() 
    
    # preprocessing (사용자 정의 all_process 함수)
    train_fold, val_fold = all_process(train_fold, val_fold)
    train2_fold, test_fold = all_process(train2_fold, test_fold)
    
    # TabularPipeline 클래스를 이용해 데이터 준비, 모델 학습 및 임베딩 추출
    pipeline = TabularPipeline(train_fold, val_fold, test_fold, seed, numeric_cols, cat_cols)
    fold_train_trans, fold_valid_trans, fold_test_trans = pipeline.run_pipeline()
    
    # LGBM용 데이터 분리 (target 컬럼 분리)
    X_train = fold_train_trans.drop(columns=['임신 성공 확률'])
    y_train = fold_train_trans['임신 성공 확률']
    X_valid = fold_valid_trans.drop(columns=['임신 성공 확률'])
    y_valid = fold_valid_trans['임신 성공 확률']
    
    # 클래스 불균형 해결을 위한 샘플 가중치 계산 (0.5 기준 이진화)
    y_train_binary = (y_train > 0.5).astype(int)
    pos_count = np.sum(y_train_binary)
    neg_count = len(y_train_binary) - pos_count
    # 양성 샘플에 부여할 가중치: 음성 수 / 양성 수 (양성이 적으면 큰 값)
    scale_pos_weight = neg_count / pos_count if pos_count > 0 else 1.0
    sample_weights = np.where(y_train_binary == 1, scale_pos_weight, 1.0)
    
    # LGBM 모델 학습 및 평가 (회귀 모델 사용)
    lgbm_params = {
        'n_estimators': 1134,
        'learning_rate': 0.009183378614268902,
        'max_depth': 15,
        'num_leaves': 59,
        'min_child_samples': 56,
        'subsample': 0.5894604069264655,
        'colsample_bytree': 0.6305670256882752,
        'reg_alpha': 7.47936987466662,
        'reg_lambda': 0.0010986427203281623
    }
    
    model_lgb = LGBMRegressor(
        **lgbm_params,
        verbosity=-1,
        n_jobs=-1,
        random_state=seed,
    )
    
    model_lgb.fit(X_train, y_train, sample_weight=sample_weights)
    
    # 예측값 생성 (회귀 모델이므로 predict 사용)
    valid_preds_proba = model_lgb.predict(X_valid)
    # 0과 1 사이로 강제 조정
    valid_preds_proba = np.clip(valid_preds_proba, 0, 1)
    valid_preds_class = (valid_preds_proba > 0.5).astype(int)

    # eval_metric.py의 사용자 정의 평가 지표 (확률 기준)
    custom_f1 = f1_score(y_valid, valid_preds_proba)
    wbs = weighted_brier_score(y_valid, valid_preds_proba)
    comp_metric = competition_metric(y_valid, valid_preds_proba)
    
    # 각 Fold별 결과 출력
    print(f"Seed[{seed:<3}] Fold {fold + 1} | Custom F1: {custom_f1:.7f} | WBS: {wbs:.7f} | Combined: {comp_metric:.7f}")
    
    f1_scores.append(custom_f1)
    wbs_scores.append(wbs)
    comp_scores.append(comp_metric)
    
    # 테스트 데이터 예측 (LGBM, predict 사용)
    test_pred = model_lgb.predict(fold_test_trans)
    # 클리핑 적용
    test_pred = np.clip(test_pred, 0, 1)
    test_preds_lgbm.append(test_pred)

    # 다음 fold를 위해 모델 가중치 초기화 (pipeline 내부의 모델 사용)
    pipeline.tabular_model.model.reset_weights()

# k-fold 종료 후, 여러 fold의 테스트 예측 평균 내기
final_test_preds = np.mean(test_preds_lgbm, axis=0)
print("Final test predictions shape:", final_test_preds.shape)

# Fold별 평균 평가 지표 출력
print("평균 Custom F1:", np.mean(f1_scores))
print("평균 WBS:", np.mean(wbs_scores))
print("평균 Combined:", np.mean(comp_scores))


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100 80GB PCIe MIG 1g.10gb') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [7]:
tmp_submission = pd.DataFrame({'embed_lgbm_nsplit_5': final_test_preds})
tmp_submission

,embed_lgbm_nsplit_5
0,0.081944
1,0.469656
2,0.409577
3,0.498812
4,0.341448
...,...
54407,0.640140
54408,0.003551
54409,0.567037
54410,0.768380


## catboost

In [3]:
seed = 333

train_path = "../../data/train.csv"
test_path = "../../data/test.csv"

train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])

In [4]:
import sys
sys.path.append("../../")
from cat_process import cb_all_process
train, test = cb_all_process(train, test)
print(train.shape, test.shape)

(126244, 33) (54412, 32)


In [6]:
cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 확률']

print(f'수치형 변수: {len(numeric_cols)}개 \n{numeric_cols}')
print(f'범주형 변수: {len(cat_cols)}개 \n{cat_cols}')
print(train.shape, test.shape)

수치형 변수: 21개 
['배란 자극 시술 여부', '단일 배아 이식 여부', '불임 원인 - 난관 질환', '불임 원인 - 배란 장애', '불임 원인 - 남성 요인', '불임 원인 - 자궁내막증', '불임 원인 - 불명확', '이전 IVF 시술 횟수', '이전 DI 시술 횟수', '이전 총 임신 횟수', '이전 총 임신 성공 횟수', '이식된 배아 수', '미세주입(ICSI) 배아 이식 수', '해동 난자 사용 여부', '신선 난자 사용 여부', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '착상 전 PGD 시행 여부', '착상 전 PGS 시행 여부', '배아 이식 후 경과일']
범주형 변수: 11개 
['환자 시술 당시 나이', '총 생성 배아 수', '저장된 배아 수', '해동된 배아 수', '채취된 신선 난자 수', '수정 시도된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '시술유형_통합']
(126244, 33) (54412, 32)


In [9]:
seed = 333
kf = KFold(n_splits=5, shuffle=True, random_state=seed)

# 학습/평가 데이터 로드 (ID 칼럼 제거)
train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

# 평가 지표들을 저장할 리스트 초기화
f1_scores = []       # Custom F1 from eval_metric
wbs_scores = []      # Weighted Brier Score
comp_scores = []     # Combined metric (competition_metric)

# 최종 예측값 저장용
test_preds_lgbm = []

# StratifiedKFold 대신 일반 KFold 사용 (타겟이 연속값인 경우 사용 가능)
for fold, (train_idx, val_idx) in enumerate(kf.split(train, train['임신 성공 확률'])):
    
    # 현재 fold의 train/validation 데이터 분할
    train_fold = train.iloc[train_idx].copy().reset_index(drop=True)
    val_fold = train.iloc[val_idx].copy().reset_index(drop=True)    
    
    train2_fold = train_fold.copy()
    test_fold = test.copy() 
    
    # preprocessing (사용자 정의 all_process 함수)
    train_fold, val_fold = cb_all_process(train_fold, val_fold)
    train2_fold, test_fold = cb_all_process(train2_fold, test_fold)
    
    # LGBM용 데이터 분리 (target 컬럼 분리)
    X_train = train_fold.drop(columns=['임신 성공 확률'])
    y_train = train_fold['임신 성공 확률']
    X_valid = val_fold.drop(columns=['임신 성공 확률'])
    y_valid = val_fold['임신 성공 확률']
    
    cat_features = X_train.select_dtypes(include=["object", "category"]).columns.tolist()
    
    # 불균형 대응용 sample_weight 생성
    y_train_binary = (y_train > 0.5).astype(int)
    pos = (y_train_binary == 1).sum()
    neg = (y_train_binary == 0).sum()
    scale_pos_weight = neg / pos if pos > 0 else 1.0
    sample_weight = np.where(y_train_binary == 1, scale_pos_weight, 1.0)
 
    
   # Catboost 모델 초기화
    model_cat = CatBoostRegressor(iterations=2000, learning_rate=0.05, random_seed=seed,
                            loss_function='Quantile:alpha=0.5', eval_metric='Quantile:alpha=0.5',
                            cat_features=cat_features, thread_count=-1)
 
    # 학습 (sample_weight 사용!)
    model_cat.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=1000, sample_weight=sample_weight)
    
    # 예측값 생성 (회귀 모델이므로 predict 사용)
    valid_preds_proba = model_cat.predict(X_valid)
    # 0과 1 사이로 강제 조정
    valid_preds_proba = np.clip(valid_preds_proba, 0, 1)
    valid_preds_class = (valid_preds_proba > 0.5).astype(int)

    # eval_metric.py의 사용자 정의 평가 지표 (확률 기준)
    custom_f1 = f1_score(y_valid, valid_preds_proba)
    wbs = weighted_brier_score(y_valid, valid_preds_proba)
    comp_metric = competition_metric(y_valid, valid_preds_proba)
    
    # 각 Fold별 결과 출력
    print(f"Seed[{seed:<3}] Fold {fold + 1} | Custom F1: {custom_f1:.7f} | WBS: {wbs:.7f} | Combined: {comp_metric:.7f}")
    
    f1_scores.append(custom_f1)
    wbs_scores.append(wbs)
    comp_scores.append(comp_metric)
    
    # 테스트 데이터 예측 (LGBM, predict 사용)
    test_pred = model_cat.predict(fold_test_trans)
    # 클리핑 적용
    test_pred = np.clip(test_pred, 0, 1)
    test_preds_lgbm.append(test_pred)

    # 다음 fold를 위해 모델 가중치 초기화 (pipeline 내부의 모델 사용)
    pipeline.tabular_model.model.reset_weights()

# k-fold 종료 후, 여러 fold의 테스트 예측 평균 내기
final_test_preds = np.mean(test_preds_lgbm, axis=0)
print("Final test predictions shape:", final_test_preds.shape)

# Fold별 평균 평가 지표 출력
print("평균 Custom F1:", np.mean(f1_scores))
print("평균 WBS:", np.mean(wbs_scores))
print("평균 Combined:", np.mean(comp_scores))


0:	learn: 0.2330861	test: 0.2248297	best: 0.2248297 (0)	total: 347ms	remaining: 11m 34s


KeyboardInterrupt: 

## 제출

In [8]:
submission = pd.read_csv('../../data/sample_submission.csv')
submission

,ID,임신 성공 확률
0,TEST_00000,0
1,TEST_00001,0
2,TEST_00002,0
3,TEST_00003,0
4,TEST_00004,0
...,...,...
54407,TEST_54407,0
54408,TEST_54408,0
54409,TEST_54409,0
54410,TEST_54410,0


In [9]:
submission['임신 성공 확률'] = final_test_preds
submission

,ID,임신 성공 확률
0,TEST_00000,0.081944
1,TEST_00001,0.469656
2,TEST_00002,0.409577
3,TEST_00003,0.498812
4,TEST_00004,0.341448
...,...,...
54407,TEST_54407,0.640140
54408,TEST_54408,0.003551
54409,TEST_54409,0.567037
54410,TEST_54410,0.768380


In [10]:
submission.to_csv('./submissions/임베딩_sampleweight_lgbm_결측값완_이산형변환.csv', index=False)